In [3]:
# Importing packages
import numpy as np
import pandas as pd 

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [4]:
# Load dataset.
telco_data = pd.read_csv('data/telco_customer_churn.csv')

# 